# SANS data reduction - DMSC Summer School

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp
import os
import utils

## Process run with sample

In [ ]:
top_folder = 'McStas_SANS_NeXus_with_issue'
run_folder = 'SANS_with_sample_NeXus_1_pulse'
fname = os.path.join(top_folder, run_folder, 'mccode.h5')

In [ ]:
sample = utils.load_nexus(fname, 100)
sample

In [ ]:
sample.hist(tof=200, y=200).plot(norm='log', vmin=1.0e-2)

In [ ]:
sample.hist(y=200).plot(norm='log')

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

graph = {**beamline(scatter=True), **elastic("tof")}
sc.show_graph(graph, simplified=True)

In [ ]:
sample.transform_coords("wavelength", graph=graph).hist(wavelength=200).plot()

In [ ]:
sample_q = sample.transform_coords("Q", graph=graph)
sample_q

In [ ]:
sample_h = sample_q.hist(Q=200)
sample_h

### Add variances to the data

In [ ]:
utils.add_variances(sample_h)
sample_h.data

In [ ]:
sample_h.plot(norm='log')

## Process flat-field run

In [ ]:
run_folder = 'SANS_without_sample_NeXus_1_pulse'
fname = os.path.join(top_folder, run_folder, 'mccode.h5')

flat = utils.load_nexus(fname, 5.0e4)

In [ ]:
flat.hist(y=200).plot()

In [ ]:
pp.plot({'sample': sample.hist(y=200),
         'flat-field': flat.hist(y=200)},
       norm='log')

In [ ]:
flat_q = flat.transform_coords("Q", graph=graph)

In [ ]:
flat_h = flat_q.hist(Q=200)
utils.add_variances(flat_h)
flat_h.plot()

## Normalize

In [ ]:
normed = sample_h / flat_h

In [ ]:
qbins = sc.linspace('Q', 5.0e-3, 0.2, 201, unit='1/angstrom')
qbins

In [ ]:
sample_h = sample_q.hist(Q=qbins)
flat_h = flat_q.hist(Q=qbins)

utils.add_variances(sample_h, flat_h)

normed = sample_h / flat_h
normed

In [ ]:
normed.plot(norm='log')

## Save result to disk

In [ ]:
q = sc.midpoints(normed.coords['Q']).values
data = np.array([q, normed.values, np.sqrt(normed.variances)]).T

np.savetxt('sans_iofq.dat', data, header="Q [1/A], I, stddev", delimiter=' ')

## Process data from 3 pulses

In [ ]:
run_folder = 'SANS_with_sample_NeXus_3_pulse'
fname = os.path.join(top_folder, run_folder, 'mccode.h5')
sample = utils.load_nexus(fname, 100)
sample

In [ ]:
sample.hist(tof=200, y=200).plot(norm='log', vmin=1.0e-2)

Convert neutron wavelength to speed:

$$ v = \frac{h}{m_{\mathrm{n}}\lambda} $$

In [ ]:
AA = sc.Unit('angstrom')
wmin = 5.25 * AA
wmax = 6.75 * AA

h = sc.constants.h
m_n = sc.constants.m_n

vmin = (h / (m_n * wmax)).to(unit='m/s')
vmax = (h / (m_n * wmin)).to(unit='m/s')
vmin, vmax

In [ ]:
mean_distance = sc.norm(sample.coords['position'].mean() - sample.coords['source_position'])
mean_distance

In [ ]:
tmin = (mean_distance / vmax).to(unit='ms')
tmax = (mean_distance / vmin).to(unit='ms')
tmin, tmax

In [ ]:
pulse_offsets = sc.arange('tof', 3.0) * sc.scalar(1.0 / 14.0, unit='s').to(unit='ms')
pulse_offsets

In [ ]:
bounds_min = tmin + pulse_offsets
bounds_max = tmax + pulse_offsets
bounds_min, bounds_max

In [ ]:
a = sc.concat([bounds_max[:-1], bounds_min[1:]], dim='dummy').mean('dummy')
a

In [ ]:
edges = sc.concat([bounds_min[0:1], a, bounds_max[-1:]], dim='tof')
edges

In [ ]:
binned = sample.bin(tof=edges)
binned

In [ ]:
binned.bins.coords['tof'] -= pulse_offsets

In [ ]:
binned

In [ ]:
binned.bins.constituents['data'].hist(tof=200, y=200).plot(norm='log', vmin=1.0e-2)